In [1]:
import sys
sys.path.insert(1, './src')

from tagger import CrfModel
from data_loader import BuildDataLoader
from environment import LabelEnv
from agent import ParamRNN, AgentParamRNN

from gensim.models import KeyedVectors
import numpy as np

import warnings; warnings.simplefilter('ignore')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.autograd import Variable  

# if gpu is to be used
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print ("== Cuda: {} ===".format(torch.cuda.is_available()))


== Cuda: True ===


In [2]:
SOURCE = 'sod'
DATA_PATH = "./datasets/" + SOURCE
NUM_FLAG = True
EMBED_FLAG = False
FEAT = 'all'

dataloader = BuildDataLoader(SOURCE, DATA_PATH, NUM_FLAG, EMBED_FLAG)
crf = CrfModel(dataloader, FEAT)

=== data loading ===
word dict size: 25
label dict size: 126
Replace digit with NUM: True
use embedding: False
embedding size: 25
=== Tagger initialized ===
-- label dict size: 126
-- word dict size: 25


In [3]:
samples = [4]
PRETRAIN_N = 15
VALID_N = 200
TEST_N = 200
REWEIGHT = 'valid2Tx'
BUDGET = 75
max_len = dataloader.get_max_len()
embedding_size = dataloader.get_embed_size()
parameter_shape = crf.get_para_shape()
print ("max_len is: {}".format(max_len))
for seed in samples:
    env = LabelEnv(dataloader, crf, seed, PRETRAIN_N, VALID_N, TEST_N, REWEIGHT, BUDGET)
    agent = AgentParamRNN(max_len, embedding_size, parameter_shape)
    
    print (">>>> Start play")
    terminal = False
    eps = 1
    while eps < 10:
        eps += 1
        observation = env.get_state()
        observ = [observation[0], observation[1], observation[3], observation[4]]
        _, action, _ = agent.get_action(observ)
        reward, observation2, terminal = env.feedback(action)
        observ2 = [observation2[0], observation2[1], observation2[3], observation2[4]]
        agent.update(observ, action, reward, observ2, terminal)
    
    print (env.acc_trace)
        

max_len is: 14
=== Env: data ===
pretrain  : 15
train     : 920
validation: 200
test      : 200
=== Env: setup ===
reward reweight: valid2Tx
similarity for valid to test done -- size=(200,)
budget: 75
start acc: 0.6382142857142846 on test, 0.6228571428571424 on valid, 0.6578483245149908 after reweight
=== Agent: created
Q-net parameter size: 2998961
>>>> Start play
tensor([[0.2484],
        [0.2320],
        [0.2185],
        [0.1962],
        [0.2630]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[0.2643],
        [0.1733],
        [0.2511],
        [0.2321],
        [0.3573]], device='cuda:0')
tensor([[7.5437],
        [8.3911],
        [8.3081],
        [8.0753],
        [4.0899]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[0.2321],
        [0.2662],
        [0.2643],
        [0.1733],
        [0.3573]], device='cuda:0')
tensor([[1.8683],
        [1.8514],
        [1.7644],
        [1.8033],
        [1.8305]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[0.2662],

KeyboardInterrupt: 

In [ ]:
a={0}
a.add(3)
print (a)